In [4]:
# A generic genetic algorithm
import random
import numpy as np
import matplotlib.pyplot as plt
from client import get_errors, get_overfit_vector, submit
from io import StringIO
from config import SECRET_KEY
from utils import POPULATION_SIZE, SPLIT_POINT, FEATURE_VECTOR_SIZE, MUTATION_COEFF
from helpers import mkdir_p, get_train_validation_errors, get_rank_and_probabilities, fitness, select_parents, cross_over, mutate, plot_graph, initialize_params, select_the_best, return_elite, list_to_array, file_name, return_order, single_mutate

from errno import EEXIST
from os import makedirs, path

In [310]:
import json
data = [[-9.62332500e-01, -2.18556820e-12, -2.72328283e-13,
         6.04360560e-11, -1.76803971e-10, -1.86060132e-15,
         8.57463659e-16,  2.29423303e-05, -2.36685403e-06,
        -1.15190527e-08,  9.61918560e-10],
       [ 7.57968557e-01, -1.45799022e-12, -1.83740854e-13,
         4.62010753e-11, -1.88047579e-10, -2.47074239e-15,
         8.52944060e-16,  2.29423303e-05, -2.22415650e-06,
        -8.19859968e-09,  9.98214034e-10],
       [-8.93553243e-01, -2.18999106e-12, -2.65421591e-13,
         6.64627450e-11, -2.28864960e-10, -6.52806729e-16,
         8.70759983e-16,  2.69769576e-05, -2.46992036e-06,
        -9.11073572e-09,  9.98214034e-10],
       [-8.82646063e-01, -3.72118162e-12, -2.78631790e-13,
         6.04360560e-11, -2.28864960e-10, -1.08626753e-15,
         8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
        -7.34988004e-09,  7.94160013e-10],
       [ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,
         4.30361039e-11, -1.88047579e-10, -1.94968615e-15,
         8.85868432e-16,  2.29423303e-05, -2.22922353e-06,
        -7.98767811e-09,  9.98214034e-10],
       [-9.62332500e-01, -1.38201793e-12, -2.68160228e-13,
         5.94850544e-11, -9.05497529e-11, -2.21302459e-15,
         8.49738440e-16,  1.65810515e-05, -2.36685403e-06,
        -2.67334581e-09,  9.61918560e-10],
       [-8.08434515e-01, -1.43528116e-12, -1.42283684e-13,
         5.07745342e-11, -1.36036036e-10, -2.28601611e-15,
         8.52944060e-16,  2.29423303e-05, -2.37988893e-06,
        -9.10981973e-09,  1.06281091e-09],
       [-8.02976518e-01, -2.02530650e-12, -3.19118882e-13,
         6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
         8.43711709e-16,  1.65810515e-05, -2.15400293e-06,
        -8.95821106e-09,  9.39560042e-10],
       [-7.35785788e-01, -1.72427876e-12, -1.42283684e-13,
         5.07745342e-11, -2.24052198e-10, -2.65273971e-15,
         8.09094056e-16,  3.01795716e-05, -1.74262678e-06,
        -9.10981973e-09,  6.20048683e-10],
       [-3.82850667e-01, -1.41047108e-12, -2.06693880e-13,
         4.62010753e-11, -1.75214813e-10, -1.83669770e-15,
         8.52944060e-16,  2.29423303e-05, -2.04721003e-06,
        -9.94388023e-09,  9.44618715e-10]]
with open('output.txt','w') as outfile:
    json.dump(data,outfile)

In [279]:
f = open('final_submission/output.txt', )
x = json.load(f)
print(x)
x = np.array(x)
output2 = np.array(x)

[[-0.9623325, -2.1855682e-12, -2.72328283e-13, 6.0436056e-11, -1.76803971e-10, -1.86060132e-15, 8.57463659e-16, 2.29423303e-05, -2.36685403e-06, -1.15190527e-08, 9.6191856e-10], [0.757968557, -1.27980381e-12, -2.28980078e-13, 4.30361039e-11, -1.88047579e-10, -1.94968615e-15, 8.85868432e-16, 2.29423303e-05, -2.22922353e-06, -7.98767811e-09, 9.98214034e-10], [0.757968557, -1.45799022e-12, -1.83740854e-13, 4.62010753e-11, -1.88047579e-10, -2.47074239e-15, 8.5294406e-16, 2.29423303e-05, -2.2241565e-06, -8.19859968e-09, 9.98214034e-10], [-0.882646063, -3.72118162e-12, -2.7863179e-13, 6.0436056e-11, -2.2886496e-10, -1.08626753e-15, 8.87078986e-16, 2.3906319e-05, -2.04721003e-06, -7.34988004e-09, 7.94160013e-10], [0.757968557, -1.27980381e-12, -2.28980078e-13, 4.30361039e-11, -1.88047579e-10, -1.94968615e-15, 8.85868432e-16, 2.29423303e-05, -2.22922353e-06, -7.98767811e-09, 9.98214034e-10], [0.757968557, -1.45799022e-12, -1.83740854e-13, 4.62010753e-11, -1.88047579e-10, -2.47074239e-15, 8.529

In [11]:
a = [-8.08434515e-01, -1.43528116e-12, -1.42283684e-13,  5.07745342e-11,
  -1.36036036e-10, -2.28601611e-15,  8.52944060e-16 , 2.29423303e-05,
  -2.37988893e-06, -9.10981973e-09 , 1.06281091e-09]
x = get_errors(SECRET_KEY, list(a))
y = submit(SECRET_KEY, list(a))
print(y)
print("{:e}".format(int(x[0])), "{:e}".format(int(x[1])))
print("Difference - {:e}".format(abs(x[0]-x[1])))
# rank 56
# 3-06 run1.txt (4th vec 4th generation)


successfully submitted
1.697112e+12 4.477850e+12
Difference - 2.780738e+12


In [19]:
a = [-7.35785788e-01, -1.72427876e-12 ,-1.42283684e-13,  5.07745342e-11,
  -2.24052198e-10, -2.65273971e-15,  8.09094056e-16,  3.01795716e-05,
  -1.74262678e-06, -9.10981973e-09,  6.20048683e-10]
x = get_errors(SECRET_KEY, list(a))
y = submit(SECRET_KEY, list(a))
print(y)
print("{:e}".format(int(x[0])), "{:e}".format(int(x[1])))
print("Difference - {:e}".format(abs(x[0]-x[1])))
# rank 70
# 3-06 run1.txt (8th vec 5th generation)

successfully submitted
3.045725e+12 1.902014e+12
Difference - 1.143711e+12


In [31]:
a = [-8.57660717e-01, -1.43528116e-12, -1.42283684e-13 , 4.51250471e-11,
  -2.42991645e-10, -2.28601611e-15,  9.40661184e-16 , 2.29423303e-05,
  -2.37988893e-06, -9.10981973e-09,  1.06281091e-09]
x = get_errors(SECRET_KEY, list(a))
y = submit(SECRET_KEY, list(a))
print(y)
print("{:e}".format(int(x[0])), "{:e}".format(int(x[1])))
print("Difference - {:e}".format(abs(x[0]-x[1])))
# rank 57
# 3-06 run1.txt (9th vec 5th generation)

successfully submitted
1.697112e+12 4.477850e+12
Difference - 2.780738e+12


In [5]:
a = [[[-8.82646063e-01, -1.40551374e-12, -2.72328283e-13,
          4.30878207e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  2.29423303e-05, -2.04721003e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.46690542e-01, -2.02530650e-12, -3.19118882e-13,
          3.91081523e-11, -1.15198504e-10, -5.86741720e-16,
          8.57463659e-16,  1.65810515e-05, -2.10774585e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -2.21430118e-12, -3.19118882e-13,
          4.51720352e-11, -1.72886683e-10, -4.46726872e-16,
          8.79345236e-16,  1.38926179e-05, -2.04721003e-06,
         -1.46729723e-08,  9.98214034e-10],
        [-8.83261626e-01, -1.87722385e-12, -3.03528324e-13,
          5.87628961e-11, -2.58282184e-10, -1.86060132e-15,
          8.87078986e-16,  2.39976097e-05, -2.20629133e-06,
         -1.89410189e-08,  9.35621799e-10],
        [-9.00543070e-01, -2.12134379e-12, -3.35746461e-13,
          6.45764292e-11, -1.62312347e-10, -1.94968615e-15,
          8.17089439e-16,  1.80781366e-05, -1.64213396e-06,
         -1.77183268e-08,  9.65266921e-10],
        [-8.02420788e-01, -3.04240243e-12, -2.78631790e-13,
          6.01051517e-11, -1.20565158e-10, -1.28456735e-15,
          7.99936532e-16,  2.48817479e-05, -1.35993388e-06,
         -2.41053257e-08,  9.18961600e-10],
        [-9.04853293e-01, -2.21430118e-12, -2.78631790e-13,
          5.45934953e-11, -1.25572443e-10, -3.97234318e-16,
          8.57463659e-16,  1.65810515e-05, -2.04721003e-06,
         -1.46729723e-08,  9.98214034e-10],
        [ 7.36207142e-01, -1.27980381e-12, -2.33352895e-13,
          3.93090895e-11, -2.75743149e-10, -2.19396909e-15,
          8.52944060e-16,  2.63838155e-05, -1.88631172e-06,
         -1.51328814e-08,  1.02765340e-09],
        [ 8.19905665e-01, -2.04283015e-12, -3.03528324e-13,
          4.79107086e-11, -2.19392432e-10, -1.94968615e-15,
          7.37238837e-16,  2.63838155e-05, -2.04721003e-06,
         -1.59792834e-08,  1.06172282e-09]],

       [[-8.82646063e-01, -1.40551374e-12, -2.72328283e-13,
          4.30878207e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  2.29423303e-05, -2.04721003e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.02976518e-01, -2.02530650e-12, -3.19118882e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.43711709e-16,  1.65810515e-05, -2.15400293e-06,
         -8.95821106e-09,  9.39560042e-10],
        [-8.00037180e-01, -2.02530650e-12, -3.57227091e-13,
          3.91081523e-11, -2.28864960e-10, -5.95330352e-16,
          7.77667316e-16,  3.06377012e-05, -2.86356748e-06,
         -1.77183268e-08,  9.81978444e-10],
        [-8.18633991e-01, -1.78822126e-12, -3.13992954e-13,
          5.86639518e-11, -2.05106169e-10, -1.94968615e-15,
          8.57463659e-16,  1.65810515e-05, -2.85062114e-06,
         -8.88104970e-09,  1.06791338e-09],
        [-9.04853293e-01, -2.12134379e-12, -2.57160835e-13,
          6.45764292e-11, -1.00871331e-10, -1.94968615e-15,
          8.17089439e-16,  1.42057790e-05, -1.64213396e-06,
         -2.34039464e-08,  1.02025022e-09],
        [-8.82646063e-01, -2.18999106e-12, -2.36072500e-13,
          3.39130823e-11, -1.15198504e-10, -6.62422000e-16,
          8.78054342e-16,  2.14356474e-05, -2.10774585e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -2.21430118e-12, -3.05064566e-13,
          5.25038571e-11, -1.72886683e-10, -4.46726872e-16,
          8.79345236e-16,  1.38926179e-05, -1.40380541e-06,
         -8.46772461e-09,  1.05574305e-09],
        [-8.82646063e-01, -1.40551374e-12, -2.65421591e-13,
          6.04360560e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  3.00792129e-05, -2.04721003e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.59763435e-01, -3.00010317e-12, -2.72968273e-13,
          6.10519205e-11, -2.81152328e-10, -1.80090014e-15,
          8.43152521e-16,  3.19576088e-05, -1.28840823e-06,
         -1.77183268e-08,  1.00715146e-09]],

       [[-8.82646063e-01, -1.40551374e-12, -2.72328283e-13,
          4.30878207e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  2.29423303e-05, -2.04721003e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-9.62332500e-01, -2.18556820e-12, -2.72328283e-13,
          6.04360560e-11, -1.76803971e-10, -1.86060132e-15,
          8.57463659e-16,  2.29423303e-05, -2.36685403e-06,
         -1.15190527e-08,  9.61918560e-10],
        [-8.82646063e-01, -3.13372297e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -2.71592279e-15,
          9.63248216e-16,  1.67705721e-05, -2.36601524e-06,
         -1.59578761e-08,  9.98214034e-10],
        [-8.90755519e-01, -1.48493957e-12, -2.72328283e-13,
          3.59134913e-11, -2.28864960e-10, -7.83530133e-16,
          8.57463659e-16,  2.56114779e-05, -1.25280025e-06,
         -1.77183268e-08,  9.67707096e-10],
        [-8.75015923e-01, -1.40551374e-12, -2.78631790e-13,
          4.30878207e-11, -2.28864960e-10, -8.43424162e-16,
          8.57463659e-16,  1.84210753e-05, -2.08004875e-06,
         -2.21080357e-08,  9.98214034e-10],
        [-8.02976518e-01, -2.02530650e-12, -3.19118882e-13,
          3.58090352e-11, -1.60247026e-10, -1.20809573e-15,
          7.60417686e-16,  2.33293526e-05, -2.15400293e-06,
         -1.12566356e-08,  9.96852000e-10],
        [-8.02976518e-01, -2.02530650e-12, -3.19118882e-13,
          6.04360560e-11, -2.28864960e-10, -2.21302459e-15,
          8.43711709e-16,  1.65810515e-05, -1.81783394e-06,
         -1.77183268e-08,  1.03608735e-09],
        [-8.82646063e-01, -2.18999106e-12, -2.36072500e-13,
          6.04360560e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  2.14356474e-05, -2.64288987e-06,
         -1.67207722e-08,  9.23015309e-10],
        [-8.93553243e-01, -2.18999106e-12, -2.65421591e-13,
          6.64627450e-11, -2.28864960e-10, -6.52806729e-16,
          8.70759983e-16,  2.69769576e-05, -2.46992036e-06,
         -9.11073572e-09,  9.98214034e-10]],

       [[-8.82646063e-01, -1.40551374e-12, -2.72328283e-13,
          4.30878207e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  2.29423303e-05, -2.04721003e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-7.30409264e-01, -2.02530650e-12, -2.43773102e-13,
          4.30878207e-11, -1.60247026e-10, -1.20809573e-15,
          8.38980229e-16,  2.35120070e-05, -2.15400293e-06,
         -1.12566356e-08,  9.98214034e-10],
        [-8.22293430e-01, -1.40551374e-12, -2.72328283e-13,
          3.05378128e-11, -1.64933483e-10, -3.49947094e-16,
          8.23028626e-16,  2.29423303e-05, -2.04721003e-06,
         -1.12566356e-08,  9.98214034e-10],
        [-9.28200603e-01, -3.04240243e-12, -2.78631790e-13,
          6.31049397e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  2.39063190e-05, -2.80393331e-06,
         -1.54381848e-08,  9.98214034e-10],
        [-8.82646063e-01, -3.04240243e-12, -2.76189646e-13,
          6.39839277e-11, -2.22977507e-10, -1.20809573e-15,
          8.44299127e-16,  2.80697862e-05, -2.04721003e-06,
         -2.24910531e-08,  1.03079392e-09],
        [-8.02976518e-01, -1.40551374e-12, -2.72328283e-13,
          6.04360560e-11, -2.28864960e-10, -5.89708719e-16,
          8.79941915e-16,  2.32593619e-05, -2.04721003e-06,
         -1.77183268e-08,  1.05900303e-09],
        [-8.02976518e-01, -6.26247500e-13, -2.72328283e-13,
          6.25203564e-11, -2.28864960e-10, -1.20809573e-15,
          7.60793500e-16,  1.65810515e-05, -2.04721003e-06,
         -1.77183268e-08,  1.03608735e-09],
        [-8.02976518e-01, -2.43172815e-12, -3.62334956e-13,
          6.14176461e-11, -1.78917283e-10, -2.21302459e-15,
          7.60152823e-16,  1.03691603e-05, -1.81783394e-06,
         -1.77183268e-08,  7.65902010e-10],
        [-9.62332500e-01, -1.38201793e-12, -2.68160228e-13,
          5.94850544e-11, -9.05497529e-11, -2.21302459e-15,
          8.49738440e-16,  1.65810515e-05, -2.36685403e-06,
         -2.67334581e-09,  9.61918560e-10]],

       [[-8.82646063e-01, -1.40551374e-12, -2.72328283e-13,
          4.30878207e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  2.29423303e-05, -2.04721003e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.82646063e-01, -3.77222962e-12, -3.20109178e-13,
          6.04360560e-11, -2.22977507e-10, -1.86060132e-15,
          9.22193902e-16,  2.80697862e-05, -1.58038027e-06,
         -8.92490563e-09,  1.03079392e-09],
        [-8.82646063e-01, -2.94015583e-12, -2.94299108e-13,
          6.04360560e-11, -2.22977507e-10, -1.11924807e-15,
          8.44299127e-16,  3.13255070e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.82646063e-01, -3.04240243e-12, -2.32989112e-13,
          6.37874339e-11, -2.28864960e-10, -1.84826486e-15,
          8.79941915e-16,  1.90951488e-05, -1.39403318e-06,
         -2.24910531e-08,  1.05900303e-09],
        [-8.82646063e-01, -2.69508936e-12, -2.72328283e-13,
          5.92248184e-11, -2.28864960e-10, -5.89708719e-16,
          9.65556233e-16,  2.32593619e-05, -2.66829966e-06,
         -1.77183268e-08,  1.05900303e-09],
        [-8.82646063e-01, -3.04240243e-12, -2.68160228e-13,
          6.39839277e-11, -2.22977507e-10, -1.80737658e-15,
          9.02918666e-16,  1.65810515e-05, -2.50413596e-06,
         -3.16009089e-09,  8.91077435e-10],
        [-8.82646063e-01, -2.15749452e-12, -2.76189646e-13,
          6.39839277e-11, -2.22977507e-10, -7.62989032e-16,
          8.49738440e-16,  2.80697862e-05, -1.31566481e-06,
         -2.37559945e-08,  9.48917304e-10],
        [-8.33069316e-01, -6.29187811e-13, -2.72328283e-13,
          4.64258977e-11, -2.28864960e-10, -1.34475442e-15,
          7.20990175e-16,  8.95621257e-06, -2.35217161e-06,
         -1.77183268e-08,  1.72516054e-10],
        [-9.35027361e-01, -6.98939569e-13, -2.72328283e-13,
          4.30878207e-11, -2.15910787e-10, -1.20809573e-15,
          7.60793500e-16,  1.07582818e-05, -2.94710965e-06,
         -2.12399518e-08,  3.72714843e-10]]]
a = np.array(a)
#x = get_errors(SECRET_KEY, list(a[5, 9, :]))
#y = submit(SECRET_KEY, list(a[4, 9, :]))
#print(y)
#print("{:e}".format(int(x[0])), "{:e}".format(int(x[1])))
#print("Difference - {:e}".format(abs(x[0]-x[1])))

In [6]:
b = [[[-8.82646063e-01, -1.40551374e-12, -2.72328283e-13,
          4.30878207e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  2.29423303e-05, -2.04721003e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.82646063e-01, -2.94015583e-12, -2.68160228e-13,
          6.39839277e-11, -3.05168992e-10, -1.34904782e-15,
          7.90679648e-16,  3.25485004e-05, -1.87338177e-06,
         -2.00363604e-08,  9.81420891e-10],
        [-8.82646063e-01, -2.94015583e-12, -2.68160228e-13,
          6.39839277e-11, -2.38207864e-10, -1.80244727e-15,
          9.68735505e-16,  3.13255070e-05, -2.50413596e-06,
         -3.16009089e-09,  9.24647083e-10],
        [-8.82646063e-01, -2.26674827e-12, -2.43268043e-13,
          7.01542457e-11, -2.02152538e-10, -7.77514035e-16,
          8.49738440e-16,  2.05450900e-05, -1.05839759e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.82646063e-01, -1.32648809e-12, -2.76189646e-13,
          6.04360560e-11, -1.73491070e-10, -1.11924807e-15,
          8.49738440e-16,  2.41641846e-05, -2.59829604e-06,
         -2.03590004e-08,  9.31860189e-10],
        [-9.28907999e-01, -8.14332342e-13, -2.44936232e-13,
          4.30878207e-11, -2.28864960e-10, -1.20809573e-15,
          8.24971831e-16,  2.49014829e-05, -1.68446374e-06,
         -9.77650433e-09,  9.74958789e-10],
        [-8.82646063e-01, -2.94015583e-12, -2.94299108e-13,
          6.04360560e-11, -2.22977507e-10, -1.11924807e-15,
          8.44299127e-16,  3.13255070e-05, -2.04721003e-06,
         -1.78188603e-08,  1.06771889e-09],
        [-8.82646063e-01, -3.67298285e-12, -2.94299108e-13,
          6.04360560e-11, -2.22977507e-10, -1.11924807e-15,
          8.44553953e-16,  3.85053301e-05, -1.69140840e-06,
         -2.97679053e-08,  1.01051837e-09],
        [-8.82646063e-01, -3.04240243e-12, -2.32989112e-13,
          6.98918643e-11, -2.28864960e-10, -1.19968627e-15,
          8.79941915e-16,  1.80446807e-05, -2.04721003e-06,
         -2.24910531e-08,  1.50662204e-09]],

       [[-8.82646063e-01, -1.40551374e-12, -2.72328283e-13,
          4.30878207e-11, -2.28864960e-10, -1.20809573e-15,
          8.57463659e-16,  2.29423303e-05, -2.04721003e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.39084496e-01, -2.94015583e-12, -2.50496764e-13,
          6.04360560e-11, -2.22977507e-10, -1.11924807e-15,
          8.14453410e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.82646063e-01, -3.93300196e-12, -3.39538104e-13,
          6.90304606e-11, -2.28864960e-10, -6.76023819e-16,
          9.63898225e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.82646063e-01, -3.04240243e-12, -3.30353966e-13,
          6.04360560e-11, -2.83877973e-10, -2.65228776e-15,
          9.41341340e-16,  2.39063190e-05, -2.04721003e-06,
         -1.77183268e-08,  9.61918560e-10],
        [-8.50140457e-01, -1.78337291e-12, -2.72328283e-13,
          4.30878207e-11, -2.28864960e-10, -1.86060132e-15,
          9.03131366e-16,  2.29423303e-05, -2.04721003e-06,
         -1.77183268e-08,  9.98214034e-10],
        [-8.82646063e-01, -1.40551374e-12, -2.72328283e-13,
          3.73036755e-11, -2.28864960e-10, -1.11924807e-15,
          8.57463659e-16,  2.29423303e-05, -1.69140840e-06,
         -2.15163839e-08,  1.81701562e-09],
        [-8.82646063e-01, -3.67298285e-12, -2.23109732e-13,
          6.04360560e-11, -1.65267896e-10, -1.11924807e-15,
          7.67780978e-16,  3.85053301e-05, -2.89245549e-06,
         -2.97679053e-08,  1.57866155e-09],
        [-8.37013953e-01, -3.80066442e-12, -3.76860213e-13,
          6.39839277e-11, -2.29009749e-10, -1.34904782e-15,
          8.44299127e-16,  3.13255070e-05, -2.04721003e-06,
         -1.86302503e-08,  1.06771889e-09],
        [-9.22217079e-01, -2.94015583e-12, -2.68160228e-13,
          6.04360560e-11, -3.50885629e-10, -1.34904782e-15,
          7.55171528e-16,  3.13255070e-05, -1.47689618e-06,
         -2.40192142e-08,  9.81420891e-10]],

       [[-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.39084496e-01, -2.94015583e-12, -2.50496764e-13,
          6.04360560e-11, -2.22977507e-10, -1.11924807e-15,
          8.14453410e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.82646063e-01, -3.93300196e-12, -1.90536527e-13,
          4.06630394e-11, -2.28864960e-10, -2.06309113e-15,
          9.13655351e-16,  2.29423303e-05, -2.24888911e-06,
         -1.77183268e-08,  9.61918560e-10],
        [-8.45812082e-01, -1.78337291e-12, -3.39538104e-13,
          3.90113210e-11, -2.70401476e-10, -1.86060132e-15,
          9.45842134e-16,  2.29423303e-05, -2.04721003e-06,
         -1.54381848e-08,  9.68370426e-10],
        [-8.82646063e-01, -3.87577120e-12, -3.39538104e-13,
          5.62876434e-11, -2.28864960e-10, -1.24439621e-15,
          8.91640547e-16,  2.79835403e-05, -2.04721003e-06,
         -1.54381848e-08,  1.00338567e-09],
        [-8.52865046e-01, -3.20213300e-12, -2.78631790e-13,
          6.04360560e-11, -1.82481417e-10, -1.65964644e-15,
          8.29508107e-16,  2.54161739e-05, -1.31074660e-06,
         -1.54381848e-08,  1.01825975e-09],
        [-8.50140457e-01, -1.78337291e-12, -2.19260784e-13,
          3.56142258e-11, -2.28864960e-10, -2.65228776e-15,
          9.03131366e-16,  2.96391389e-05, -1.45231390e-06,
         -1.05426464e-08,  9.98214034e-10],
        [-8.10834379e-01, -3.04240243e-12, -3.30353966e-13,
          6.04360560e-11, -2.70565093e-10, -2.65228776e-15,
          9.41341340e-16,  2.29423303e-05, -2.48424942e-06,
         -1.77183268e-08,  9.66850521e-10],
        [-8.39084496e-01, -2.94015583e-12, -3.00754544e-13,
          5.84478191e-11, -2.28864960e-10, -1.85397207e-15,
          9.47121752e-16,  2.19689054e-05, -2.04721003e-06,
         -1.93760708e-08,  9.98214034e-10],
        [-8.37496683e-01, -1.40551374e-12, -2.72328283e-13,
          5.43612208e-11, -2.22977507e-10, -1.20809573e-15,
          8.14453410e-16,  2.44695477e-05, -2.04721003e-06,
         -2.38325038e-08,  1.00836127e-09]],

       [[-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.39084496e-01, -2.94015583e-12, -2.50496764e-13,
          6.04360560e-11, -2.22977507e-10, -1.11924807e-15,
          8.14453410e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-9.20940160e-01, -3.34089383e-12, -3.68203093e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.90551465e-05, -1.39754716e-06,
         -2.18617217e-08,  9.61918560e-10],
        [-7.63281422e-01, -2.82166387e-12, -2.94761633e-13,
          6.67553759e-11, -2.77047918e-10, -1.85397207e-15,
          8.87078986e-16,  2.07473013e-05, -2.80199766e-06,
         -2.09420377e-08,  9.98214034e-10],
        [-8.93661911e-01, -3.87577120e-12, -2.97642648e-13,
          4.85365289e-11, -2.83232867e-10, -1.08626753e-15,
          8.91640547e-16,  2.18421359e-05, -2.04721003e-06,
         -1.54381848e-08,  1.00338567e-09],
        [-8.39084496e-01, -2.68740781e-12, -2.83042447e-13,
          5.45903143e-11, -2.10287582e-10, -1.85397207e-15,
          9.24125333e-16,  2.19689054e-05, -2.04721003e-06,
         -2.74483489e-08,  1.08131334e-09],
        [-8.82646063e-01, -3.04240243e-12, -3.53407887e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.52583479e-16,  2.51349933e-05, -2.04721003e-06,
         -2.38325038e-08,  9.61918560e-10],
        [-8.37496683e-01, -2.19483971e-12, -3.48977146e-13,
          5.43612208e-11, -2.75549384e-10, -1.86060132e-15,
          8.14453410e-16,  2.44695477e-05, -2.08844429e-06,
         -1.53804827e-08,  9.37862350e-10],
        [-7.75938191e-01, -9.16376932e-13, -2.78631790e-13,
          3.94098375e-11, -3.36635818e-10, -1.19673353e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -7.73904459e-09,  9.08573320e-10],
        [-8.45812082e-01, -3.04240243e-12, -3.39538104e-13,
          6.04360560e-11, -2.28864960e-10, -1.28959462e-15,
          8.08855931e-16,  2.29423303e-05, -2.04721003e-06,
         -2.10006043e-08,  1.00657588e-09]],

       [[-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.39084496e-01, -2.94015583e-12, -2.50496764e-13,
          6.04360560e-11, -2.22977507e-10, -1.11924807e-15,
          8.14453410e-16,  2.39063190e-05, -2.04721003e-06,
         -1.54381848e-08,  9.61918560e-10],
        [-8.93661911e-01, -3.87577120e-12, -2.97642648e-13,
          4.85365289e-11, -2.64471098e-10, -1.40887032e-15,
          9.47051861e-16,  2.18421359e-05, -2.57607052e-06,
         -1.53804827e-08,  1.03424077e-09],
        [-8.37496683e-01, -2.19483971e-12, -3.12289172e-13,
          5.43612208e-11, -2.75549384e-10, -2.41715231e-15,
          8.42083537e-16,  2.44695477e-05, -2.04721003e-06,
         -1.53804827e-08,  9.37862350e-10],
        [-8.82646063e-01, -3.72118162e-12, -2.78631790e-13,
          6.04360560e-11, -2.28864960e-10, -1.08626753e-15,
          8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
         -7.34988004e-09,  7.94160013e-10],
        [-8.93661911e-01, -3.72241519e-12, -3.04865891e-13,
          5.15889922e-11, -2.57255120e-10, -1.11404770e-15,
          8.40417963e-16,  1.50198276e-05, -2.04721003e-06,
         -1.97082711e-08,  1.11206326e-09],
        [-7.89117523e-01, -2.19039732e-12, -2.97642648e-13,
          6.04360560e-11, -2.22977507e-10, -4.44020538e-16,
          8.07231572e-16,  2.39063190e-05, -2.04721003e-06,
         -1.51354985e-08,  1.48696663e-09],
        [-8.04397122e-01, -3.50325907e-12, -2.97642648e-13,
          6.04360560e-11, -2.83232867e-10, -1.47795346e-15,
          8.91640547e-16,  2.18421359e-05, -2.04721003e-06,
         -1.67833897e-08,  1.02469977e-09],
        [-8.90542067e-01, -1.79712005e-12, -3.48977146e-13,
          6.48504007e-11, -2.75549384e-10, -1.86060132e-15,
          8.14453410e-16,  2.44695477e-05, -2.20886146e-06,
         -1.91829809e-08,  9.37862350e-10],
        [-9.19500859e-01, -2.94015583e-12, -2.15784695e-13,
          4.68567363e-11, -1.77867077e-10, -2.35085372e-15,
          7.27733929e-16,  2.44695477e-05, -1.48867558e-06,
         -1.67591964e-08,  9.61918560e-10]]]
b = np.array(b)
#x = get_errors(SECRET_KEY, list(a[0, 2, :]))
#y = submit(SECRET_KEY, list(a[0, 2, :]))
#print(y)
#print("{:e}".format(int(x[0])), "{:e}".format(int(x[1])))
#print("Difference - {:e}".format(abs(x[0]-x[1])))

In [282]:
a[1, 2]

array([-8.02976518e-01, -2.02530650e-12, -3.19118882e-13,  6.04360560e-11,
       -2.28864960e-10, -1.86060132e-15,  8.43711709e-16,  1.65810515e-05,
       -2.15400293e-06, -8.95821106e-09,  9.39560042e-10])

In [7]:
output1 = np.zeros(shape=(10, 11), dtype=float)
output1[0, :] = a[2, 2, :]
output1[1, :] = a[2, 9, :]
output1[2, :] = a[3, 9, :]
output1[3, :] = b[4, 4, :]
output1[4, :] = np.array([ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,  4.30361039e-11,
  -1.88047579e-10, -1.94968615e-15,  8.85868432e-16 , 2.29423303e-05,
  -2.22922353e-06, -7.98767811e-09,  9.98214034e-10])
output1[5, :] = np.array([ 7.57968557e-01, -1.45799022e-12, -1.83740854e-13,  4.62010753e-11,
  -1.88047579e-10, -2.47074239e-15 , 8.52944060e-16,  2.29423303e-05,
  -2.22415650e-06, -8.19859968e-09 , 9.98214034e-10])
output1[6, :] = [-8.08434515e-01, -1.43528116e-12, -1.42283684e-13,  5.07745342e-11,
  -1.36036036e-10, -2.28601611e-15,  8.52944060e-16 , 2.29423303e-05,
  -2.37988893e-06, -9.10981973e-09 , 1.06281091e-09]
output1[7, :] = a[1, 2, :]
output1[8, :] = np.array([-7.35785788e-01, -1.72427876e-12 ,-1.42283684e-13,  5.07745342e-11,
  -2.24052198e-10, -2.65273971e-15,  8.09094056e-16,  3.01795716e-05,
  -1.74262678e-06, -9.10981973e-09,  6.20048683e-10])
output1[9, :] = np.array([-3.82850667e-01, -1.41047108e-12, -2.06693880e-13,  4.62010753e-11,
  -1.75214813e-10, -1.83669770e-15,  8.52944060e-16,  2.29423303e-05,
  -2.04721003e-06, -9.94388023e-09,  9.44618715e-10])
# run3.txt
# a[0, 0] rank 70
# a[0, 3] rank 54 done
# a[1, 7] rank 53 done
# a[3, 9] rank 77
# Continuing run3 - run4.txt
# a[1, 2] rank 67 -
# a[2, 2] rank 49 -
# a[2, 6] rank 74
# a[2, 9] rank 52 -
# a[3, 2] rank 77
# a[3, 9] rank 52 -
# Continuing run4 - run4.txt
# a[4, 4] rank 52 done
output1

array([[-9.62332500e-01, -2.18556820e-12, -2.72328283e-13,
         6.04360560e-11, -1.76803971e-10, -1.86060132e-15,
         8.57463659e-16,  2.29423303e-05, -2.36685403e-06,
        -1.15190527e-08,  9.61918560e-10],
       [-8.93553243e-01, -2.18999106e-12, -2.65421591e-13,
         6.64627450e-11, -2.28864960e-10, -6.52806729e-16,
         8.70759983e-16,  2.69769576e-05, -2.46992036e-06,
        -9.11073572e-09,  9.98214034e-10],
       [-9.62332500e-01, -1.38201793e-12, -2.68160228e-13,
         5.94850544e-11, -9.05497529e-11, -2.21302459e-15,
         8.49738440e-16,  1.65810515e-05, -2.36685403e-06,
        -2.67334581e-09,  9.61918560e-10],
       [-8.82646063e-01, -3.72118162e-12, -2.78631790e-13,
         6.04360560e-11, -2.28864960e-10, -1.08626753e-15,
         8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
        -7.34988004e-09,  7.94160013e-10],
       [ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,
         4.30361039e-11, -1.88047579e-10, -1.94968615e-15,
  

In [277]:
#x = get_errors(SECRET_KEY, list(output[5]))
#y = submit(SECRET_KEY, list(output[5]))
#print(y)
#print("{:e}".format(int(x[0])), "{:e}".format(int(x[1])))
#ranks:
# 49 0
# 51 1
# 51 2
# 52 3
# 52 4
# 53 5
# 53 6
# 68 7
# 71 8
# 69 9
a[3, 9]

array([-9.62332500e-01, -1.38201793e-12, -2.68160228e-13,  5.94850544e-11,
       -9.05497529e-11, -2.21302459e-15,  8.49738440e-16,  1.65810515e-05,
       -2.36685403e-06, -2.67334581e-09,  9.61918560e-10])

In [272]:
temp = output[8]
output[8] = output[9]
output[9] = temp
output[5] = output1[2]

In [291]:
output - output1

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.65152180e+00,  9.10187250e-13,  3.64415130e-14,
        -2.34266411e-11,  4.08173810e-11, -1.29687942e-15,
         1.51084490e-17, -4.03462730e-06,  2.40696830e-07,
         1.12305761e-09,  0.00000000e+00],
       [ 1.72030106e+00, -7.59722900e-14,  8.44193740e-14,
        -1.32839791e-11, -9.74978261e-11, -2.57717800e-16,
         3.20562000e-18,  6.36127880e-06,  1.42697530e-07,
        -5.52525387e-09,  3.62954740e-11],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
  

In [294]:
x = get_errors(SECRET_KEY, list(output1[8]))
y = submit(SECRET_KEY, list(output1[8]))
print(y)
print("{:e}".format(int(x[0])), "{:e}".format(int(x[1])))
#59 1
#62 2
# 58 5
# 71 8

# 6
#'''
#[-9.62332500e-01, -1.38201793e-12, -2.68160228e-13,
#         5.94850544e-11, -9.05497529e-11, -2.21302459e-15,
#         8.49738440e-16,  1.65810515e-05, -2.36685403e-06,
#        -2.67334581e-09,  9.61918560e-10]
#'''

successfully submitted
3.045725e+12 1.902014e+12


In [289]:
output1

array([[-9.62332500e-01, -2.18556820e-12, -2.72328283e-13,
         6.04360560e-11, -1.76803971e-10, -1.86060132e-15,
         8.57463659e-16,  2.29423303e-05, -2.36685403e-06,
        -1.15190527e-08,  9.61918560e-10],
       [-8.93553243e-01, -2.18999106e-12, -2.65421591e-13,
         6.64627450e-11, -2.28864960e-10, -6.52806729e-16,
         8.70759983e-16,  2.69769576e-05, -2.46992036e-06,
        -9.11073572e-09,  9.98214034e-10],
       [-9.62332500e-01, -1.38201793e-12, -2.68160228e-13,
         5.94850544e-11, -9.05497529e-11, -2.21302459e-15,
         8.49738440e-16,  1.65810515e-05, -2.36685403e-06,
        -2.67334581e-09,  9.61918560e-10],
       [-8.82646063e-01, -3.72118162e-12, -2.78631790e-13,
         6.04360560e-11, -2.28864960e-10, -1.08626753e-15,
         8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
        -7.34988004e-09,  7.94160013e-10],
       [ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,
         4.30361039e-11, -1.88047579e-10, -1.94968615e-15,
  

In [280]:
output2

array([[-9.62332500e-01, -2.18556820e-12, -2.72328283e-13,
         6.04360560e-11, -1.76803971e-10, -1.86060132e-15,
         8.57463659e-16,  2.29423303e-05, -2.36685403e-06,
        -1.15190527e-08,  9.61918560e-10],
       [ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,
         4.30361039e-11, -1.88047579e-10, -1.94968615e-15,
         8.85868432e-16,  2.29423303e-05, -2.22922353e-06,
        -7.98767811e-09,  9.98214034e-10],
       [ 7.57968557e-01, -1.45799022e-12, -1.83740854e-13,
         4.62010753e-11, -1.88047579e-10, -2.47074239e-15,
         8.52944060e-16,  2.29423303e-05, -2.22415650e-06,
        -8.19859968e-09,  9.98214034e-10],
       [-8.82646063e-01, -3.72118162e-12, -2.78631790e-13,
         6.04360560e-11, -2.28864960e-10, -1.08626753e-15,
         8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
        -7.34988004e-09,  7.94160013e-10],
       [ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,
         4.30361039e-11, -1.88047579e-10, -1.94968615e-15,
  

In [305]:
np.copyto(output3, output1)
output3[1, :] = output1[5, :]
output3[2] = output1[1]
output3[5] = output1[2]

In [307]:
output3 - output1

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.65152180e+00,  7.32000840e-13,  8.16807370e-14,
        -2.02616697e-11,  4.08173810e-11, -1.81793566e-15,
        -1.78159230e-17, -4.03462730e-06,  2.45763860e-07,
         9.12136040e-10,  0.00000000e+00],
       [ 6.87792570e-02, -8.07973130e-13,  2.73863700e-15,
         6.97769060e-12, -1.38315207e-10,  1.56021786e-15,
         2.10215430e-17,  1.03959061e-05, -1.03066330e-07,
        -6.43738991e-09,  3.62954740e-11],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
  

In [302]:
output1

array([[-9.62332500e-01, -2.18556820e-12, -2.72328283e-13,
         6.04360560e-11, -1.76803971e-10, -1.86060132e-15,
         8.57463659e-16,  2.29423303e-05, -2.36685403e-06,
        -1.15190527e-08,  9.61918560e-10],
       [ 7.57968557e-01, -1.45799022e-12, -1.83740854e-13,
         4.62010753e-11, -1.88047579e-10, -2.47074239e-15,
         8.52944060e-16,  2.29423303e-05, -2.22415650e-06,
        -8.19859968e-09,  9.98214034e-10],
       [ 7.57968557e-01, -1.45799022e-12, -1.83740854e-13,
         4.62010753e-11, -1.88047579e-10, -2.47074239e-15,
         8.52944060e-16,  2.29423303e-05, -2.22415650e-06,
        -8.19859968e-09,  9.98214034e-10],
       [-8.82646063e-01, -3.72118162e-12, -2.78631790e-13,
         6.04360560e-11, -2.28864960e-10, -1.08626753e-15,
         8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
        -7.34988004e-09,  7.94160013e-10],
       [ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,
         4.30361039e-11, -1.88047579e-10, -1.94968615e-15,
  

In [308]:
output3

array([[-9.62332500e-01, -2.18556820e-12, -2.72328283e-13,
         6.04360560e-11, -1.76803971e-10, -1.86060132e-15,
         8.57463659e-16,  2.29423303e-05, -2.36685403e-06,
        -1.15190527e-08,  9.61918560e-10],
       [ 7.57968557e-01, -1.45799022e-12, -1.83740854e-13,
         4.62010753e-11, -1.88047579e-10, -2.47074239e-15,
         8.52944060e-16,  2.29423303e-05, -2.22415650e-06,
        -8.19859968e-09,  9.98214034e-10],
       [-8.93553243e-01, -2.18999106e-12, -2.65421591e-13,
         6.64627450e-11, -2.28864960e-10, -6.52806729e-16,
         8.70759983e-16,  2.69769576e-05, -2.46992036e-06,
        -9.11073572e-09,  9.98214034e-10],
       [-8.82646063e-01, -3.72118162e-12, -2.78631790e-13,
         6.04360560e-11, -2.28864960e-10, -1.08626753e-15,
         8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
        -7.34988004e-09,  7.94160013e-10],
       [ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,
         4.30361039e-11, -1.88047579e-10, -1.94968615e-15,
  

In [ ]:
output1 = np.zeros(shape=(10, 11), dtype=float)
output1[0, :] = a[2, 2, :]
output1[1, :] = a[2, 9, :]
output1[2, :] = a[3, 9, :]
output1[3, :] = b[4, 4, :]
output1[4, :] = np.array([ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,  4.30361039e-11,
  -1.88047579e-10, -1.94968615e-15,  8.85868432e-16 , 2.29423303e-05,
  -2.22922353e-06, -7.98767811e-09,  9.98214034e-10])
output1[5, :] = np.array([ 7.57968557e-01, -1.45799022e-12, -1.83740854e-13,  4.62010753e-11,
  -1.88047579e-10, -2.47074239e-15 , 8.52944060e-16,  2.29423303e-05,
  -2.22415650e-06, -8.19859968e-09 , 9.98214034e-10])
output1[6, :] = [-8.08434515e-01, -1.43528116e-12, -1.42283684e-13,  5.07745342e-11,
  -1.36036036e-10, -2.28601611e-15,  8.52944060e-16 , 2.29423303e-05,
  -2.37988893e-06, -9.10981973e-09 , 1.06281091e-09]
output1[7, :] = a[1, 2, :]
output1[8, :] = np.array([-7.35785788e-01, -1.72427876e-12 ,-1.42283684e-13,  5.07745342e-11,
  -2.24052198e-10, -2.65273971e-15,  8.09094056e-16,  3.01795716e-05,
  -1.74262678e-06, -9.10981973e-09,  6.20048683e-10])
output1[9, :] = np.array([-3.82850667e-01, -1.41047108e-12, -2.06693880e-13,  4.62010753e-11,
  -1.75214813e-10, -1.83669770e-15,  8.52944060e-16,  2.29423303e-05,
  -2.04721003e-06, -9.94388023e-09,  9.44618715e-10])
# run3.txt
# a[0, 0] rank 70
# a[0, 3] rank 54 done
# a[1, 7] rank 53 done
# a[3, 9] rank 77
# Continuing run3 - run4.txt
# a[1, 2] rank 67 -
# a[2, 2] rank 49 -
# a[2, 6] rank 74
# a[2, 9] rank 52 -
# a[3, 2] rank 77
# a[3, 9] rank 52 -
# Continuing run4 - run4.txt
# a[4, 4] rank 52 done

In [2]:
import numpy as np
output_final = np.array([[-0.9623325, -2.1855682e-12, -2.72328283e-13, 6.0436056e-11, -1.76803971e-10, -1.86060132e-15, 8.57463659e-16, 2.29423303e-05, -2.36685403e-06, -1.15190527e-08, 9.6191856e-10], [0.757968557, -1.45799022e-12, -1.83740854e-13, 4.62010753e-11, -1.88047579e-10, -2.47074239e-15, 8.5294406e-16, 2.29423303e-05, -2.2241565e-06, -8.19859968e-09, 9.98214034e-10], [-0.893553243, -2.18999106e-12, -2.65421591e-13, 6.6462745e-11, -2.2886496e-10, -6.52806729e-16, 8.70759983e-16, 2.69769576e-05, -2.46992036e-06, -9.11073572e-09, 9.98214034e-10], [-0.882646063, -3.72118162e-12, -2.7863179e-13, 6.0436056e-11, -2.2886496e-10, -1.08626753e-15, 8.87078986e-16, 2.3906319e-05, -2.04721003e-06, -7.34988004e-09, 7.94160013e-10], [0.757968557, -1.27980381e-12, -2.28980078e-13, 4.30361039e-11, -1.88047579e-10, -1.94968615e-15, 8.85868432e-16, 2.29423303e-05, -2.22922353e-06, -7.98767811e-09, 9.98214034e-10], [-0.9623325, -1.38201793e-12, -2.68160228e-13, 5.94850544e-11, -9.05497529e-11, -2.21302459e-15, 8.4973844e-16, 1.65810515e-05, -2.36685403e-06, -2.67334581e-09, 9.6191856e-10], [-0.808434515, -1.43528116e-12, -1.42283684e-13, 5.07745342e-11, -1.36036036e-10, -2.28601611e-15, 8.5294406e-16, 2.29423303e-05, -2.37988893e-06, -9.10981973e-09, 1.06281091e-09], [-0.802976518, -2.0253065e-12, -3.19118882e-13, 6.0436056e-11, -2.2886496e-10, -1.86060132e-15, 8.43711709e-16, 1.65810515e-05, -2.15400293e-06, -8.95821106e-09, 9.39560042e-10], [-0.735785788, -1.72427876e-12, -1.42283684e-13, 5.07745342e-11, -2.24052198e-10, -2.65273971e-15, 8.09094056e-16, 3.01795716e-05, -1.74262678e-06, -9.10981973e-09, 6.20048683e-10], [-0.382850667, -1.41047108e-12, -2.0669388e-13, 4.62010753e-11, -1.75214813e-10, -1.8366977e-15, 8.5294406e-16, 2.29423303e-05, -2.04721003e-06, -9.94388023e-09, 9.44618715e-10]])

In [17]:
output_final[9]

array([-3.82850667e-01, -1.41047108e-12, -2.06693880e-13,  4.62010753e-11,
       -1.75214813e-10, -1.83669770e-15,  8.52944060e-16,  2.29423303e-05,
       -2.04721003e-06, -9.94388023e-09,  9.44618715e-10])

In [8]:
output1

array([[-9.62332500e-01, -2.18556820e-12, -2.72328283e-13,
         6.04360560e-11, -1.76803971e-10, -1.86060132e-15,
         8.57463659e-16,  2.29423303e-05, -2.36685403e-06,
        -1.15190527e-08,  9.61918560e-10],
       [-8.93553243e-01, -2.18999106e-12, -2.65421591e-13,
         6.64627450e-11, -2.28864960e-10, -6.52806729e-16,
         8.70759983e-16,  2.69769576e-05, -2.46992036e-06,
        -9.11073572e-09,  9.98214034e-10],
       [-9.62332500e-01, -1.38201793e-12, -2.68160228e-13,
         5.94850544e-11, -9.05497529e-11, -2.21302459e-15,
         8.49738440e-16,  1.65810515e-05, -2.36685403e-06,
        -2.67334581e-09,  9.61918560e-10],
       [-8.82646063e-01, -3.72118162e-12, -2.78631790e-13,
         6.04360560e-11, -2.28864960e-10, -1.08626753e-15,
         8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
        -7.34988004e-09,  7.94160013e-10],
       [ 7.57968557e-01, -1.27980381e-12, -2.28980078e-13,
         4.30361039e-11, -1.88047579e-10, -1.94968615e-15,
  

In [ ]:
[[3.36646232e+12, 4.26656929e+12],
[2.51184563e+12, 6.81699296e+12],
[1.47955702e+12, 1.41400146e+12],
[6.35487008e+11, 2.29566016e+11],
[3.59808204e+12, 8.72027872e+12],
[1.38537801e+12, 2.71484142e+12],
[1.697112e+12, 4.477850e+12],
[2.42323104e+11, 3.18716579e+11],
[3.045725e+12, 1.902014e+12],
[2.41400100e+12, 6.48112577e+12]]

In [33]:
a = None
a = [[-8.82646063e-01, -8.39084496e-01, -8.45812082e-01, -8.82646063e-01,
  -8.39084496e-01, -8.37496683e-01],
 [-3.04240243e-12, -2.94015583e-12, -1.78337291e-12, -3.87577120e-12,
  -2.94015583e-12, -1.40551374e-12],
 [-2.78631790e-13, -2.50496764e-13, -3.39538104e-13, -3.39538104e-13,
  -3.00754544e-13, -2.72328283e-13],
 [ 6.04360560e-11,  6.04360560e-11,  3.90113210e-11,  5.62876434e-11,
   5.84478191e-11,  5.43612208e-11],
 [-2.28864960e-10, -2.22977507e-10, -2.70401476e-10, -2.28864960e-10,
  -2.28864960e-10, -2.22977507e-10],
 [-1.86060132e-15, -1.11924807e-15, -1.86060132e-15, -1.24439621e-15,
  -1.85397207e-15, -1.20809573e-15],
 [ 8.87078986e-16,  8.14453410e-16,  9.45842134e-16 , 8.91640547e-16,
   9.47121752e-16,  8.14453410e-16],
 [ 2.39063190e-05,  2.39063190e-05,  2.29423303e-05,  2.79835403e-05,
   2.19689054e-05,  2.44695477e-05],
 [-2.04721003e-06, -2.04721003e-06, -2.04721003e-06, -2.04721003e-06,
  -2.04721003e-06, -2.04721003e-06],
 [-1.54381848e-08, -1.54381848e-08, -1.54381848e-08, -1.54381848e-08,
  -1.93760708e-08, -2.38325038e-08],
 [ 9.61918560e-10,  9.61918560e-10,  9.68370426e-10,  1.00338567e-09,
   9.98214034e-10,  1.00836127e-09]]
b = np.transpose(np.array(a))
#a = list(b)
b

array([[-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
         6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
         8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
        -1.54381848e-08,  9.61918560e-10],
       [-8.39084496e-01, -2.94015583e-12, -2.50496764e-13,
         6.04360560e-11, -2.22977507e-10, -1.11924807e-15,
         8.14453410e-16,  2.39063190e-05, -2.04721003e-06,
        -1.54381848e-08,  9.61918560e-10],
       [-8.45812082e-01, -1.78337291e-12, -3.39538104e-13,
         3.90113210e-11, -2.70401476e-10, -1.86060132e-15,
         9.45842134e-16,  2.29423303e-05, -2.04721003e-06,
        -1.54381848e-08,  9.68370426e-10],
       [-8.82646063e-01, -3.87577120e-12, -3.39538104e-13,
         5.62876434e-11, -2.28864960e-10, -1.24439621e-15,
         8.91640547e-16,  2.79835403e-05, -2.04721003e-06,
        -1.54381848e-08,  1.00338567e-09],
       [-8.39084496e-01, -2.94015583e-12, -3.00754544e-13,
         5.84478191e-11, -2.28864960e-10, -1.85397207e-15,
  

In [32]:
b = np.transpose(np.array(a))
b

array([[-8.82646063e-01, -1.40551374e-12, -2.72328283e-13,
         4.30878207e-11, -2.28864960e-10, -1.20809573e-15,
         8.57463659e-16,  2.29423303e-05, -2.04721003e-06,
        -1.77183268e-08,  9.98214034e-10],
       [-8.82646063e-01, -3.04240243e-12, -2.78631790e-13,
         6.04360560e-11, -2.28864960e-10, -1.86060132e-15,
         8.87078986e-16,  2.39063190e-05, -2.04721003e-06,
        -1.54381848e-08,  9.61918560e-10],
       [-8.39084496e-01, -2.94015583e-12, -2.50496764e-13,
         6.04360560e-11, -2.22977507e-10, -1.11924807e-15,
         8.14453410e-16,  2.39063190e-05, -2.04721003e-06,
        -1.54381848e-08,  9.61918560e-10],
       [-8.82646063e-01, -3.93300196e-12, -3.39538104e-13,
         6.90304606e-11, -2.28864960e-10, -6.76023819e-16,
         9.63898225e-16,  2.39063190e-05, -2.04721003e-06,
        -1.54381848e-08,  9.61918560e-10],
       [-8.82646063e-01, -3.04240243e-12, -3.30353966e-13,
         6.04360560e-11, -2.83877973e-10, -2.65228776e-15,
  

In [ ]:
Red, Orange, Blue, Yellow, Green Purple